# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [ ]:
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

In [98]:
import Crypto.Util.number as cu
import Crypto.Random as cr
import hashlib as hl
import math

**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.  --> funkcja Eulera

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

d * e == 1 w ciele f

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [99]:
nlen = 2048

def generate_keys():
    p = cu.getStrongPrime(nlen//2)
    q = cu.getStrongPrime(nlen//2)
    # cond 3, 4, 5
    while p <= pow(2, (nlen-1)//2) or q <= pow(2, (nlen-1)//2) or abs(p-q) <= pow(2, nlen/2 - 100):
        p = cu.getStrongPrime(nlen//2)
        q = cu.getStrongPrime(nlen//2)
        
    n = p * q
    f = (p-1) * (q-1)
    
    # cond 1, 2
    e = cu.getRandomRange(65537, min(f, pow(2, 256)))
    while cu.GCD(f, e) != 1 or math.lcm(p-1, q-1) < e * pow(2,nlen//2):
         e = cu.getRandomRange(65537, min(f, pow(2, 256)))  
    
    d = pow(e, -1, f)
    
    return (e, d, n)

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


***Czy M może być dowolna?*** 
Musi spełniać warunek: jaki?


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [117]:
M = "Ala ma kota"

# Your code here
def naive_encrypt(m, e, n):
    mybytes = m.encode('utf-8')
    c = pow(int.from_bytes(mybytes, byteorder='big'), e, n)
    return c.to_bytes(math.ceil(c.bit_length() / 8), byteorder='big')

def naive_decrypt(c, d, n):
    m = pow(int.from_bytes(c, byteorder='big'), d, n)
    mbytes = m.to_bytes(math.ceil(m.bit_length() / 8), byteorder='big')
    return mbytes.decode('utf-8')

E, D, N = generate_keys()
encrypted_M = naive_encrypt(M, E, N)
print("ENCRYPTED MESSAGE:")
print(encrypted_M)
decrypted_M = naive_decrypt(encrypted_M, D, N)
print("DECRYPTED MESSAGE:")
print(decrypted_M)

ENCRYPTED MESSAGE:
b'~\xafi2\x18(\xbbsp,?-\x06\xfe\xf7\x10\x1c\xab>\xa4\x87\\E[\x91\x154\xc7\x84\xd6\x14-\xafh$\xa6\x04b\xd8@?\xe2\x84F9J\xb1\xa8\x80\x8a\xd9\x15\x98\xe5"\x00\xc1\x16\x9c\xae(\x1c\x12\xcdq\xfe\x93\xaf\x90i\x97f\x02\xf5O<\x94\xc2\x0c\xb6\xa0\xbc\xbf(\x89\x97\xed\x1d\xa4\xae\xa0\x89\x11\x14\xd6\x05\x03x\x8d\x9b\x07\xc4\xe9#T\xfd*r\xec\x82\xea\xf3d\x12\x1bb\xeb\xe1\xf8!\xd87\x853|\xfc3\x1c\xff\\\xc1\xfa;\xf9q\x90\xbe[\r\xbb\xeb\xda\xa3Yn\xf5wV\xbfQB\xad\xf4\x81!\x0f\xb2\xe53z\xef\xa6\xae\x80(,zn%\x99\x88]J\xb1\x0f\x1a\xa2$ug\xa8A#!\xc6\x06M\xfc=2\xf3b\x1b\xb0e\xb2Ur^0}\xa7\x0cvjaA\x10\xfd\xacS\xa2\x7f\xa6\xd7\xa5\xcb\x8e\x7f\x16\xa5\x15\xfa\x0bgbF-\xf7v\xf8\x18\xc7=<\xd8w\x90\x01\x06\xf0\xe4\xfd`\xda9\xa0\xce\x18\xc5R\xcb^\xaff\xe6'
DECRYPTED MESSAGE:
Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [12]:
hl.sha256("test".encode('utf-8')).digest()

b'\x9f\x86\xd0\x81\x88L}e\x9a/\xea\xa0\xc5Z\xd0\x15\xa3\xbfO\x1b+\x0b\x82,\xd1]l\x15\xb0\xf0\n\x08'

In [101]:
# Your code here
def naive_create_signature(m, d, n):
    hbytes = hl.sha256(m.encode('utf-8')).digest()
    h = int.from_bytes(hbytes, byteorder='big')
    sig = pow(h, d, n)
    return (m, sig)

def naive_verify_signature(m, sig, e, n):
    hbytes = hl.sha256(m.encode('utf-8')).digest()
    h = int.from_bytes(hbytes, byteorder='big')
    ver = pow(sig, e, n)
    if ver == h:
        return 'positive verification'
    return 'negative verification'

MESS, SIG = naive_create_signature(M, D, N)
print("signature:\n", SIG)
naive_verify_signature(MESS, SIG, E, N)

signature:
 16954200850299596581193072007354547381093877443673254867235498306765185869494436470457910470429156131116970699207823247609607989553190698146174866252085963141767585988503962327544313725783186018088302399913861575995989211679579592286223123831436597072223140769009726021570055157795838471421175735135655551013163346975061749142315665516553136622876420934084464681571629030025710914920658916733076027698316758290280141240072276934720426719472607291088432862911820746079532016559307933548400781886716898350000710084264114785916537293909509534577985069703937943644358496393076466244198737188363805915492135664790131977200


'positive verification'

## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [102]:
hlen = 32

def MGF1(m, size, h=hl.sha256):
    output = bytearray()
    for i in range(0, math.ceil(size/hlen)):
        ibytes = i.to_bytes(4, 'big')
        tmp = h(m + ibytes).digest()
        output = output + tmp
    return output[: size + 1]

# check with the wikipedia example
MGF1(b"foo", 3, hl.sha1)

bytearray(b'\x1a\xc9\x07\\')

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [155]:
M = "Ala ma kota"

def encode_OAEP(k, mlen, m, mgf1, h=hl.sha256):
    if mlen > k - 2 * hlen - 2:
        raise ValueError("mlen > k - 2 * hlen - 2")
    lHash = h(b"").digest()
    ps = bytearray(k - mlen - 2 * hlen - 2)
    db = lHash + ps + b'\x01' + m
    print(len(db), ' should equal: ', k - hlen - 1)
    seed = cr.get_random_bytes(hlen)
    dbMask = mgf1(seed, k - hlen - 1)
    maskedDB = bytearray([db[i] ^ dbMask[i] for i in range(len(db))])
    seedMask = mgf1(maskedDB, hlen)
    maskedSeed = bytearray([seed[i] ^ seedMask[i] for i in range(len(seed))])
    em = b'\x00' + maskedSeed + maskedDB
    print(len(em), ' should equal: ', k)
    return em

def decode_OAEP(k, mgf1, em, h=hl.sha256):
    y = em[0]
    maskedSeed = em[1:hlen+1]
    maskedDB = em[hlen+1:]
    lHash = h(b"").digest()
    seedMask = mgf1(maskedDB, hlen)
    seed = bytearray([maskedSeed[i] ^ seedMask[i] for i in range(len(maskedSeed))])
    dbMask = mgf1(seed, k - hlen - 1)
    db = bytearray([maskedDB[i] ^ dbMask[i] for i in range(len(maskedDB))])
    
    lHash2 = db[:hlen]
    db = db[hlen:]

    i = 0
    while db[i] != 1: 
        i += 1
    
    if db[i] == 1: 
        i += 1
    
    m = db[i:]
    
    if y != 0:
        raise ValueError("y should equal 0x00")
    if lHash != lHash2:
        raise ValueError("lHash and lHash' should be equal", lHash, lHash2)
    return m

K = 78
encoded_M = encode_OAEP(K, len(M.encode('utf-8')), M.encode('utf-8'), MGF1)
print("ENCODED MESSAGE:")
print(encoded_M)
decoded_M = decode_OAEP(K, MGF1, encoded_M)
print("DECODED MESSAGE:")
print(decoded_M.decode())

45  should equal:  45
78  should equal:  78
ENCODED MESSAGE:
b'\x00F\x16\xcd\xcb#\x1e`\x81S\\\xbbvQ\x16\x14>\xfbd\x87v\xf9\x9a\xf7\xad4\xa9\x007*\xe4C\xd6`\xdby\xaeB`\x95\xdd\x8eiz}\xc0#\xbe\x91\xa7\x9dbc\xe1\xd5\xeb\x1d2\x00\x80y\xe2#\xf90\x82\xa2R\x8b\xf8xG\xcc\xf8\xd6\xd06\x17'
DECODED MESSAGE:
Ala ma kota


In [165]:
def OAEP_encrypt(m, e, n, mgf1):
    k = math.ceil(n.bit_length() / 8)
#     mbytes = m.encode('utf-8')
    mbytespadded = encode_OAEP(k, len(m), m, mgf1)
    c = pow(int.from_bytes(mbytespadded, byteorder='big'), e, n)
    return c.to_bytes(k, byteorder='big')

def OAEP_decrypt(c, d, n, mgf1):
    k = math.ceil(n.bit_length() / 8)
    m = pow(int.from_bytes(c, byteorder='big'), d, n)
    mbytes = m.to_bytes(k, byteorder='big')
    return decode_OAEP(k, mgf1, mbytes)

E, D, N = generate_keys()
encrypted_M = OAEP_encrypt(M.encode('utf-8'), E, N, MGF1)
print("ENCRYPTED MESSAGE:")
print(encrypted_M)
decrypted_M = OAEP_decrypt(encrypted_M, D, N, MGF1)
print("DECRYPTED MESSAGE:")
print(decrypted_M)

223  should equal:  223
256  should equal:  256
ENCRYPTED MESSAGE:
b"\xbc?N>=\xc9\xf7\x9a\xd6+-eM\xc1\xa3\x07_s\x11\x8er\xcf\xcb\xf7\x15 \x80F\x0c\x1eRWL*d\xec\x90{\xab\x1b\xde\xd1\x8d\xf2\xae\x89\x86'|\xd6[-\xb4\xedS0\xdd\x87\x8fX\xcf\xc7\x94\x9f\xa5\xaf\xeb?\xbc\x88\xb3\xc3\x12>\x0f\x0c\xd1d\x1e)\xc7\x10\xd7\xe7<\xf5\x16n0\xdc\x19G\x15U\xee\xd2\x85\xd5\xdb\x90\xb7\xca-Y\xbf\xd7\xaf\xcdK\x98'U\x03e\xeb^]\x80\xb2t\x92\x96\xf9\xe4\xdc\x0b\x962G\xc0\xad\xf2\x86\xfd\x08x\xbdJ9UN\x97\x85\xdda&9\xc0:4\xc6\xf6\x90\x94\xe9\x87lCF\xe4\xef\xbf\x0e@\xeb\x93U!\xf2e\x81\xa7\xa9\xd6\xa5T\xc0\x00\x16`Y\xea\xe0A\x18,\xfa53\x9b\xda\x84\xe5k\x85\xfa\xf30.\x8ffw\x87\xd4m\x92\x08V~\xec{\xcdh\xb16\xdc\xb0k\xb6\xf8\x9a\xe3\x98G\xc6\xa3\n\x13\xc65w5v\x8f\xff\xa2d\xab\xb0/Ci0{\xb5\xad\xdc\xe2\x9f\xd8\xae\x9c\xe5TKK"
DECRYPTED MESSAGE:
bytearray(b'Ala ma kota')


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [125]:
m = "Ala ma podpis"

def encode_EMSA(slen, m, mgf1, emBits, emlen, h=hl.sha256):
    mHash = h(m).digest()
    if emlen < hlen + slen + 2:
        raise ValueError('emlen < hlen + slen + 2')
    salt = cr.get_random_bytes(slen)
    m2 = b'\x00' * 8 + mHash + salt
    print(len(m2), 'should equal ', 8 + hlen + slen)
    mHash2 = h(m2).digest()
    ps = bytearray(emlen - slen - hlen - 2)
    db = ps + b'\x01' + salt
    print(len(db), 'should equal ', emlen - hlen - 1)
    dbMask = mgf1(mHash2, emlen - hlen - 1)
    maskedDB = bytearray([db[i] ^ dbMask[i] for i in range(len(db))])
    maskedDB[:8 * emlen - emBits] = b'\x00' * (8 * emlen - emBits)
    em = maskedDB + mHash2 + b'\xbc'
    return em

def decode_EMSA(slen, em, m, mgf1, emBits, emlen, h=hl.sha256):
    mHash = h(m).digest()
    if emlen < hlen + slen + 2:
        raise ValueError('emlen < hlen + slen + 2')
    if em[-1] != 188:
        return('1: VERIFICATION NEGATIVE')
    maskedDB = em[: emlen - hlen - 1]
    em = em[emlen - hlen - 1:]
    
    mHash2 = em[:hlen]
    em = em[hlen:]
    if maskedDB[:8 * emlen - emBits] != b'\x00' * (8 * emlen - emBits):
        return('2: VERIFICATION NEGATIVE')
    dbMask = mgf1(mHash2, emlen - hlen - 1)
    db = bytearray([maskedDB[i] ^ dbMask[i] for i in range(len(maskedDB))])
    db[: 8 * emlen - emBits] = b'\x00' * (8 * emlen - emBits)
    if db[:emlen - hlen - slen - 2] != b'\x00' * (emlen - hlen - slen - 2) or db[emlen - hlen - slen - 2] != 1:
        return('3: VERIFICATION NEGATIVE')
    salt = db[-slen:]
    m2 = b'\x00' * 8 + mHash + salt
    print(len(m2), 'should be equal to', 8 + hlen + slen)
    mHash3 = h(m2).digest()
    if(mHash2 != mHash3):
        return('4: VERIFICATION NEGATIVE')
    return('VERIFICATION POSITIVE')
        
encoded_M = encode_EMSA(hlen, M.encode('utf-8'), MGF1, nlen - 1 , nlen//8)
print("SIGNATURE:")
print(encoded_M)
decoded_M = decode_EMSA(hlen, encoded_M, M.encode('utf-8'), MGF1, nlen - 1, nlen//8)
decoded_M

72 should equal  72
223 should equal  223
SIGNATURE:
bytearray(b'\x00\x0f;E\x1b&\xae\x9a\x9c\xbc2w\xe3\x83%\xe3\x01\xa2"\xb1\xb1Xk\t_||\xe0\xcf\xba\xc2\x94v\xff\x10\x12fs\xd9\xd1G\x84\xe5\xa6\x9b5=V\xd6|L\x1eBr9I\xe0\x1f\x99\xc0\xf2\xcbw\xb9\x15R\xf2[^\xffc$\xc9\xf3"\xa9\xeb\xb3N\xb9\xe0\xf4\xafP7\xb0J\xcb\x12\xf4QX\xe5\xc8\x9d\x1a\x11\x16\xa6z./G\xb9h\x1f\xc6\xb0\x1e\xf3\xe3\x8c\x00\xf77\xbc\xda\r\x01)l|\x86p_\xdf\x01\xdfE\x86d\xf2\x8c\x8a\x9czJ\xfbV\x12\xf5w41\x98\xc4\xf8\xee\x80\x892\xe0\xd1\xddZ\xa1\xb3c\xd0\xcbm\'\x00.\xdc\xd5W\x7fe9x\n\xadxS*\xe1\xc9\x0f\x81\xda}M}\x90/\xaar\xa9\x1a\xcduL\xc4\xba\xd4\xad\xf4\xe2v\x0f\x82\xf1\xd0\x03\xe3\xa26!\xb0k\xbdP(Le\x04T)k\x9c\x1bF\xe8hK\x10\x85?@\x00\xc6\\\xe9\xc0\xc9\x9fv\x126\x12\x17\xe3\xcc\x91\x85\xd8\x83moT\xfd\xf5\xf9\xdb\xbc')
72 should be equal to 72


'VERIFICATION POSITIVE'

In [133]:
# Your code here
def EMSA_create_signature(m, d, n, mgf1):
    em = encode_EMSA(hlen, m.encode('utf-8'), mgf1, nlen - 1 , nlen//8)
    h = int.from_bytes(em, byteorder='big')
    sig = pow(h, d, n)
    return (m, sig)

def EMSA_verify_signature(m, sig, e, n, mgf1):
    ver = pow(sig, e, n)
    encoded_M = ver.to_bytes(nlen//8, byteorder='big')
    h = decode_EMSA(hlen, encoded_M, m.encode('utf-8'), mgf1, nlen - 1, nlen//8)
    return h

MESS, SIG = EMSA_create_signature(M, D, N, MGF1)
print("signature:\n", SIG)
EMSA_verify_signature(MESS, SIG, E, N, MGF1)

72 should equal  72
223 should equal  223
signature:
 7155650621589448627039926888571344979632189723717535076113557863869942250773548700709251470742601417443501313417010704535642702380151754627872395861864427583117670890370485358297107134682306274824736352203269298164614531307174307567185168353309434103511802008789527092370214499652479898625704761724840094007901109968116636874137827098730530431268868332609899148768330927193192680781807597743065820367837356430772249826713861964950359938717463018640692776560244487790730361339541351701597943921288439927470820964077945644001035371044930446936853175540662592336847180595358718739851401474593305004254899863396976018594
72 should be equal to 72


'VERIFICATION POSITIVE'

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [173]:
def encryptKEM(e, n, mgf1, h=hl.sha256):
    rand = cr.get_random_bytes(1)
    key = h(rand).digest()
    print("KEY")
    print(key)
    ciphered_key = OAEP_encrypt(key, e, n, mgf1)
    return ciphered_key

def decryptKEM(ciphered_key, d, n, mgf1, h=hl.sha256):
    key = OAEP_decrypt(ciphered_key, d, n, mgf1)
    return key

E, D, N = generate_keys()
ciphered_key = encryptKEM(E, N, MGF1)
print(decryptKEM(ciphered_key, D, N, MGF1))
ciphered_key

KEY
b'>\xbe\x1bYv*\x1c\x80 \xc1\xef\xe3t}\xd0\x7f\x0e0a~\xd6\x0bNj[\xee\x16\xb6\xeaB\x1d\xd0'
223  should equal:  223
256  should equal:  256
bytearray(b'>\xbe\x1bYv*\x1c\x80 \xc1\xef\xe3t}\xd0\x7f\x0e0a~\xd6\x0bNj[\xee\x16\xb6\xeaB\x1d\xd0')


b'2\xcd\x0b->\xf9\xa0 \xe8[\x9d\x08\xca\xdd\x81\x03\x00\xfd\xe3`Q\x96\x80T\x86\x91?\xe7\xecT<\xedM+\x9d\xcb\x0c\x9b}\xb6J\x04d\xe3J\xaa\x02\xde\x95#Z\xb9\x9a\x8e\xc1\xa9\xc5\x17\x9bS\xa2\x9a\xa4@\xda\x9cr\x8e\xb0\x1c\xd1tC\x0e\x0e\x1a\x91\x11\xfbE\x17\x88^Qv\xb4\xa1\xa9S\xf8\xe0|\xa7/\xbf\xc1$V\xb9\xa4\x1c\xba\xc9Dg\xfb\xf5T\xc2\xc9\x19\xc4gz\xc1!#\xc0ts.\x93j\xaa K\xcb\x89VFub\xcel\xd4=\x1e\xf872\xe1^\x05\t\x16\xcd\xde\x8dZ\xb7\xb5v\xe4\x04^\x9dL-\x9c\xe1\xfaV"\x1d\x8d\xc8\xcf\xfb9=\xf6OW\xdc\x1d\xbf\xeb\xa8U;\xc7\xed\xda\x07\xdc\x1c\xd8\xbbl\xcb\xf9\xe9\xd1\x8c\x18\xb8\x90k\xa8`l\xb8W\xd7Yw\xe9\xa2\x87\x8f3M\xd8k\r\xf3.\xe2N\xfb\xd4\\\xbc\xd4\xb9\xf4\x9d!qc\xd2\x87\xd1\xd3\xe0<8 \xd5\x16\xc5\x84\xe9\xa6\xff\\\xeb\x01\x08\x8e\xff_\x95\xe6m\xc2'